In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tnrange


# Import Spike Trains

In [2]:
def is_float(string):
    """ True if given string is float else False"""
    try:
        return float(string)
    except ValueError:
        return False

data = []
with open("/media/hdd01/sklee/hglm/biophysical/inputs/Ispikes_d48000_r1_rep1_Ni120_i20_I30.dat", 'r') as f:
    d = f.readlines()
    for i in d:
        k = i.rstrip().split(" ")
        data.append([float(i) if is_float(i) else i for i in k]) 

data = np.array(data, dtype='O')

In [3]:
for i in range(data.shape[0]):
    if data[i,0] == "0":
        data[i,0] = 0

print(data.shape)
bad_idx = np.where(data[:,1] == "NA")
print(bad_idx[0].size)
data = np.delete(data, bad_idx[0], axis=0)

print(data.shape)
        
print(np.max(data[:,0]))
print(np.max(data[:,1]))

data = data[np.argsort(data[:,1])]

for i in range(data.shape[0]):
    data[i,0] = int(data[i,0])

(544834, 2)
0
(544834, 2)
119.0
47999.2930629354


In [4]:
np.save("/media/hdd01/sklee/hglm/biophysical/inputs/Ispikes_d48000_r1_rep1_Ni120_i20_I30.npy", data)

# Make Neural Matrix

In [2]:
spike_train = np.load("/media/hdd01/sklee/hglm/biophysical/inputs/Ispikes_d48000_r1_rep1_Ni120_i20_I30.npy", allow_pickle=True)
print(spike_train.shape)

(544834, 2)


In [3]:
bin_size = 1 # in milliseconds
syn_no = int(np.max(spike_train[:,0])) + 1
bin_no = int(int(np.max(spike_train[:,1]) + 1) / bin_size)
neural = np.zeros((bin_no, syn_no))

bin_edges = np.arange(0, int(np.max(spike_train[:,1])) + 1 + bin_size, bin_size)
print(bin_edges.shape)
print(bin_edges)

hist_counts, hist_edges = np.histogram(spike_train[:,1], bin_edges)
print(np.sum(hist_counts))
print(spike_train.shape[0])
print(hist_counts.shape)

count = 0

for i in tnrange(hist_counts.shape[0]):
    bin_count = hist_counts[i]
    for j in range(bin_count):
        syn = spike_train[count,0]
        neural[i, syn] += 1
        count += 1
        
print(count)
print(np.sum(neural))
print(neural.shape)
print(np.nonzero(neural)[0].size/neural.size)

(48001,)
[    0     1     2 ... 47998 47999 48000]
544834
544834
(48000,)


<ipython-input-3-b130f6996fdb>:17: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(hist_counts.shape[0]):



544834
544834.0
(48000, 120)
0.029084027777777778


In [4]:
np.save("/media/hdd01/sklee/hglm/biophysical/inputs/Ispikes_d48000_r1_rep1_Ni120_i20_I30_neural.npy", neural)